Download data from openDAP to compute DOC flux

In [1]:
"""Import packages"""
import datetime
import xarray as xr
import numpy as np
import scipy.io as sio

In [ ]:
"""Get openDAP URL and download data"""
url='http://tds.marine.rutgers.edu/thredds/dodsC/roms/ecb/ches/chesroms_ecb_1980-2014/avg' 

ds = xr.open_dataset(url)

# slicing throught the time, xi_u and eta_u dimensions
Huon =ds.sel(ocean_time=slice('2006','2011'),xi_u=59,eta_u=slice(10, 25)).Huon
Huon_semilabileDOC =ds.sel(ocean_time=slice('2006','2011'),xi_u=59,eta_u=slice(10, 25)).Huon_semilabileDOC
Huon_refractoryDOC =ds.sel(ocean_time=slice('2006','2011'),xi_u=59,eta_u=slice(10, 25)).Huon_refractoryDOC
Huon_DOC=Huon_semilabileDOC+Huon_refractoryDOC

In [ ]:
"""Daily Huon*DOC"""
DOCslb1 =ds.sel(ocean_time=slice('2006','2011'),xi_rho=59,eta_rho=slice(10, 25)).semilabileDOC
DOCslb2 =ds.sel(ocean_time=slice('2006','2011'),xi_rho=60,eta_rho=slice(10, 25)).semilabileDOC
DOCslb = 0.5*(DOCslb1 + DOCslb2)
DOCrfr1 =ds.sel(ocean_time=slice('2006','2011'),xi_rho=59,eta_rho=slice(10, 25)).refractoryDOC
DOCrfr2 =ds.sel(ocean_time=slice('2006','2011'),xi_rho=60,eta_rho=slice(10, 25)).refractoryDOC
DOCrfr = 0.5*(DOCrfr1 + DOCrfr2)
DOC=DOCslb + DOCrfr

In [ ]:
"""Daily salinity and temperature"""
salt = ds.sel(ocean_time=slice('2006','2011'),xi_rho=59,eta_rho=slice(10, 25)).salt
temp= ds.sel(ocean_time=slice('2006','2011'),xi_rho=59,eta_rho=slice(10, 25)).temp

In [ ]:
def make_dataset(*data_array_list):
    xr_ds = xr.Dataset({data_array.name: data_array for
data_array in data_array_list 
                        })
    return xr_ds

In [ ]:
makeds = make_dataset(Huon, Huon_DOC, DOC, salt, temp)
makeds.to_netcdf('chesapeake_bay_docflux.nc')